# DATA COLLECTION


In [6]:
import os
import mediapipe as mp
import cv2


mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
detector = mp.solutions.hands.Hands(static_image_mode= False, min_detection_confidence= 0.5, min_tracking_confidence= 0.5, max_num_hands= 2)


def calcBoundaryBox(landmark_list, h, w):
    space = 25
    x_coord = []
    y_coord = []
    for landmark in landmark_list:
        x_coord.append(landmark.x)
        y_coord.append(landmark.y)

    return round(min(x_coord) * w) - space, round(min(y_coord) * h) - space, round(max(x_coord) * w) + space, round(max(y_coord) * h) + space


def draw_hands(imgae):
    height, width = imgae.shape[:-1]

    hand_landmarks = detector.process(imgae)

    if hand_landmarks.multi_hand_landmarks:
        for num, hand in enumerate(hand_landmarks.multi_hand_landmarks):
            xmin, ymin, xmax, ymax = calcBoundaryBox(hand.landmark, height, width)
            return imgae[xmin:xmax , ymin:ymax] ,cv2.rectangle(imgae, (xmin-20, ymin-20), (xmax+20, ymax+20), (0, 0, 0), 4), (xmin, ymin, xmax, ymax)
    else:
        return None , None , None




DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

categories = {  0: "A",
                1: "B",
                2: "C",
                3: "D",
                4: "E",
                5: "F",
                6: "G",
                7: "H",
                8: "I",
                9: "K",
                10: "L",
                11: "O",
                12: "c",
                13: "W",
                14:"Y",
                15:"5"
                }
dataset_size = 150
number_of_classes = 15

cap = cv2.VideoCapture(0)
for j in range(number_of_classes):
    if not os.path.exists(os.path.join(DATA_DIR, categories[j])):
        os.makedirs(os.path.join(DATA_DIR, categories[j]))

    print('Collecting data for class {}'.format(categories[j]))

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        frame = cv2.flip(frame, 1)
        cv2.imshow('frame', frame)
        if cv2.waitKey(40) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        hand_bbox = frame
        img_crop ,hand_bbox , coords = draw_hands(hand_bbox)
        if hand_bbox is not None:
            display = hand_bbox
            cv2.imwrite(os.path.join(DATA_DIR, categories[j], '{}.jpg'.format(counter)), img_crop)
        else:
            display = frame
        cv2.imshow('frame', display)
        cv2.waitKey(40)

        counter += 1

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 